In [1]:
import pandas as pd
import os
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.models import Sequential

#1. Cloning the GitHub repository

In [2]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 43.17 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


#2. Setting up images for the ImageDataGenerator class

In [3]:
legend = pd.read_csv('/content/facial_expressions/data/legend.csv')

replaceDict = {'HAPPINESS': 'happiness', 'SURPRISE': 'surprise', 'ANGER':'anger', 'DISGUST':'disgust', 'NEUTRAL':'neutral', 'SADNESS':'sadness', 'FEAR':'fear'}
legend['emotion'].replace(to_replace = replaceDict, value = None, inplace = True)

data = {}
for i in range(len(legend)):
  key = legend.iloc[i]['emotion']
  if key in data:
    data[key].append(legend.iloc[i]['image'])
  else:
    data[key] = [legend.iloc[i]['image']]

In [4]:
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [5]:
for emo in data.keys():
  os.mkdir(f'master_data/training/{emo}')
  os.mkdir(f'master_data/testing/{emo}')

In [6]:
for emo, image_list in data.items():
  split = int(0.8 * len(image_list))
  training_images = image_list[:split]
  testing_images = image_list[split:]

  for img in training_images:
    source = os.path.join('/content/facial_expressions/images', img)
    destination = os.path.join('/content/master_data/training', emo, img)
    copyfile(source, destination)

  for img in testing_images:
    source = os.path.join('/content/facial_expressions/images', img)
    destination = os.path.join('/content/master_data/testing', emo, img)
    copyfile(source, destination)

#3. Creating the CNN Model

In [7]:
model = Sequential()

In [8]:
model.add(Conv2D(16, (3, 3), activation = 'relu', input_shape = (100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(8, activation = 'softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 64)       0

In [10]:
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])

#4. Training and Testing the model

In [11]:
training_directory = '/content/master_data/training'
testing_directory = '/content/master_data/testing'

In [12]:
training_datagen = ImageDataGenerator(rescale = 1.0/255)
training_flow = training_datagen.flow_from_directory(
    training_directory,
    target_size = (100, 100),
    class_mode = 'categorical',
    batch_size = 64
)

testing_datagen = ImageDataGenerator(rescale = 1.0/255)
testing_flow = testing_datagen.flow_from_directory(
    testing_directory,
    target_size = (100, 100),
    class_mode = 'categorical',
    batch_size = 64
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [13]:
early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 2, min_delta = 0.01)

In [14]:
model.fit(
    training_flow,
    epochs = 20,
    verbose = 1,
    validation_data = testing_flow
)

Epoch 1/20
171/171 [==============================] - 29s 104ms/step - loss: 1.0622 - accuracy: 0.5266 - val_loss: 0.9525 - val_accuracy: 0.6149
Epoch 2/20
171/171 [==============================] - 16s 96ms/step - loss: 0.8180 - accuracy: 0.7053 - val_loss: 0.9163 - val_accuracy: 0.6506
Epoch 3/20
171/171 [==============================] - 17s 97ms/step - loss: 0.7531 - accuracy: 0.7362 - val_loss: 0.9741 - val_accuracy: 0.6233
Epoch 4/20
171/171 [==============================] - 17s 99ms/step - loss: 0.7277 - accuracy: 0.7487 - val_loss: 1.0647 - val_accuracy: 0.5890
Epoch 5/20
171/171 [==============================] - 17s 100ms/step - loss: 0.7005 - accuracy: 0.7602 - val_loss: 1.1643 - val_accuracy: 0.6298
Epoch 6/20
171/171 [==============================] - 16s 96ms/step - loss: 0.7042 - accuracy: 0.7550 - val_loss: 1.0667 - val_accuracy: 0.6309
Epoch 7/20
171/171 [==============================] - 17s 97ms/step - loss: 0.7183 - accuracy: 0.7544 - val_loss: 1.1087 - val_accurac